In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical, image_dataset_from_directory
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

In [ ]:
data = image_dataset_from_directory("drive/MyDrive/data1000", seed=123 ,batch_size=32, image_size=(256,256) )

Found 2008 files belonging to 2 classes.


In [ ]:
class_names = data.class_names
class_names

['cat', 'dog']

In [ ]:
for image_batch, label_batch in data:
  print(image_batch.shape)
  print(label_batch.numpy())  # numpy sayesinde array şeklinde yazdırdık.
# len(image_batch) # image_batch 32 uzunluğunda her birinin içersinde 68 tane fotoğraf olan değişkendir.
# len(data)


In [ ]:
# plt.figure(figsize=(10,10)) # derleyici çalıştırdığımızda gösterilecek fotoğrafın o anki boyutunu ayarlıyoruz
# for image_batch, label_match in data.take(1): # data adlı veri setinden 1 tane grup çekiyor.
#   for i in range(4):  # 12 tane fotoğraf yüklemesi için 12 olarak range ayarlıyor
#     ax = plt.subplot(2,2,i+1) # (3,4)'lük bir matris ayarlıyor. i+1 ile indeksleme yaparak subplot numarasını belirler.
#     plt.imshow(image_batch[i].numpy().astype("uint8")) # resimler numpy dizisi olarak elde edilir. uint8 türüne dönüştürülerek gösterilir.
#     plt.title(class_names[label_batch[i]]) # resimin adı class_names adlı listeye göre alır.
#     plt.axis("off")  # bununla eksen boyutlarını kapatırız.

In [ ]:
len(data)

In [ ]:
train_size = 0.8
len(data)*train_size

In [ ]:
train_data = data.take(50)
len(train_data)

In [ ]:
test_data = data.skip(50)
len(test_data)

In [ ]:
val_size = 0.1
len(data)*val_size

In [ ]:
val_data = test_data.take(6)
len(val_data)

In [ ]:
test_data = test_data.skip(6)
len(test_data)

In [ ]:
# def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
#     assert (train_split + test_split + val_split) == 1

#     ds_size = len(ds)

#     if shuffle:
#         ds = ds.shuffle(shuffle_size, seed=12)

#     train_size = int(train_split * ds_size)
#     val_size = int(val_split * ds_size)

#     traindata = ds.take(train_size)
#     val_data = ds.skip(train_size).take(val_size)
#     test_data = ds.skip(train_size).skip(val_size)

#     return train_data, val_data, test_data

In [ ]:
# train_ds, val_ds, test_ds = get_dataset_partitions_tf(data)


In [ ]:
print(len(data))
print(len(train_data))
print(len(test_data))
print(len(val_data))

In [ ]:
# train_data = train_data.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
# val_data = val_data.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
# test_data = test_data.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
resizing_rescaling = Sequential([
    layers.Resizing(256,256),
    layers.Rescaling(1./255)
])

In [ ]:
data_augmentation = Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2)
])

In [ ]:
# var olan datasetimizi büyütmek, çeşitlendirdik
train_data = train_data.map(
    lambda x,y: (data_augmentation(x, training = True), y)
).prefetch(buffer_size = tf.data.AUTOTUNE)

In [ ]:
# MODEL OLUŞTURMA
model = models.Sequential([
  resizing_rescaling,
  layers.Conv2D(32, kernel_size = (3,3), activation = "relu", input_shape = (32,256,256,3)),
  layers.MaxPooling2D(2,2),
  layers.Conv2D(32, kernel_size = (3,3), activation = "relu"),
  layers.MaxPooling2D(2,2),
  layers.Flatten(),
  layers.Dense(64, activation = "relu"),
  layers.Dense(2, activation = "softmax")
])
model.build(input_shape = (32,256,256,3))

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer = Adam(),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
    metrics = ["accuracy"]
)

In [ ]:
history = model.fit(
    train_data,
    batch_size = 32,
    validation_data = val_data,
    verbose=1,
    epochs=10,
)

In [ ]:
score = model.evaluate(test_data,  verbose=1) # verbose parametresi debug modda asşağıdaki ve yukarıdaki görüntüyü görmek için yazarız
print("test loss:", score[0])
print("test accuracy:", score[1])

In [ ]:
score

In [ ]:
model.save("cat_dog_detection.h5")

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

In [ ]:
model = load_model("cat_dog_detection.h5")


In [ ]:
for image_batch, label_batch in test_data.take(1):
  f_image = image_batch[1].numpy().astype("uint8")
  f_label = label_batch[1].numpy().astype("uint8")

  print("first image to predict")
  plt.imshow(f_image)
  print("actual label: ",class_names[f_label])
  batch_prediction = model.predict(image_batch)
  print("predicted label: ",class_names[np.argmax(batch_prediction[0])])
  plt.axis("off")

In [ ]:
image = load_img("kopek.jpg",target_size=(256,256))
image = img_to_array(image)/255
image = np.expand_dims(image, axis=0)

In [ ]:
image.shape

In [ ]:
pred = model.predict(image)

In [ ]:
pred

In [ ]:
pred.argmax()

In [ ]:
class_names[0]